In [14]:
import requests
import numpy as np
import os
import json
import plotly.express as px
import pandas as pd
pd.set_option('display.max_colwidth', 400)

import re

# NLP
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')   # Required for tokenization
nltk.download('wordnet') # Required for lemmatization
nltk.download('stopwords') # Required for lemmatization

from nltk.corpus import stopwords

from wordcloud import WordCloud
from collections import Counter

import warnings
warnings.filterwarnings("ignore")

from tqdm.notebook import tqdm
tqdm.pandas()

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# Instantiate
lemmatizer = WordNetLemmatizer()

# Exclusion list of punctuations and numbers
exclist = string.punctuation + string.digits


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\daire\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\daire\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\daire\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
metaData = pd.read_excel("MetaOfEarningCallsV2.xlsx")

In [12]:
# Create a Function
def clean_texts(text):
    """ Function to perform preprocessing """

    #remove tags
    text=re.sub("</?.*?>"," <> ",text)
    
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    # Convert to lower cases
    text = text.lower()
    
    # Remove punctuations and numbers
    text = text.translate(str.maketrans("", "", exclist))
    
    # Replace certain words
    text = text.replace("leased", "lease")
    
    # Tokenization
    tokens = word_tokenize(text)
        
    # Lemmatization
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Remove stop words
    tokens = [token for token in tokens if token not in stopwords.words('english')]
    
    # Join tokens
    clean_text = " ".join(tokens)
    
    # Return the output
    return clean_text


In [15]:
metaData["cleanText"] = metaData["Text"].progress_apply(clean_texts)

  0%|          | 0/7379 [00:00<?, ?it/s]

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

def get_stop_words(stop_file_path):
    """load stop words """
    
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return frozenset(stop_set)

#load a set of stop words
stopwords=get_stop_words("../resources/stopwords.txt")
docs=metaData["cleanText"].tolist()

#create a vocabulary of words, 
#ignore words that appear in 85% of documents, 
#eliminate stop words
cv=CountVectorizer(max_df=0.8,stop_words=list(stopwords))
word_count_vector=cv.fit_transform(docs)

In [17]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

In [18]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results


In [21]:
#generate tf-idf for all documents in your list. docs_test has 500 documents
tf_idf_vector=tfidf_transformer.transform(cv.transform(metaData["cleanText"].tolist()))
feature_names=cv.get_feature_names_out()

results=[]
for i in range(tf_idf_vector.shape[0]):
    
    # get vector for a single document
    curr_vector=tf_idf_vector[i]
    
    #sort the tf-idf vector by descending order of scores
    sorted_items=sort_coo(curr_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,20)
    
    
    results.append(keywords)

df=pd.DataFrame(zip(docs,results),columns=['cleanText','keywords_vals'])
data=metaData.merge(df, on="cleanText", how = "inner")


In [23]:
data["Keywords"] = data["keywords_vals"].progress_apply(lambda x: list(x.keys()))
flat_list = [item for sublist in list(data["Keywords"]) for item in sublist]

common_words = pd.DataFrame(Counter(flat_list).most_common(60))
common_words.columns = ('Word', 'Count')

  0%|          | 0/7379 [00:00<?, ?it/s]

In [24]:
data

,Unnamed: 0,gltr_0,gltr_1,gltr_2,gltr_3,gltr_Label,gpt_zero_Burtiness,gpt_zero_Label,gpt_zero_Perplexity,roberta_Label,roberta_large_Label,roberta_large_score_0,roberta_large_score_1,roberta_score_Fake,roberta_score_Real,Text,cleanText,keywords_vals,Keywords
0,0,0.748011,0.188329,0.055703,0.007958,1,70,1,16,0,0,0.932329,0.067671,0.000259,0.999741,"Thank you. Good afternoon and thanks for joining us. Speaking today is Apple CFO Peter Oppenheimer and he’ll be joined by Apple COO Tim Cook and Treasurer Gary Wipfler for the Q&A session with analysts. Please note that some of the information you’ll hear during our discussion today may consist of forward looking statements regarding revenue, gross margin, operating expenses, other income and ...",thank good afternoon thanks joining u speaking today apple cfo peter oppenheimer joined apple coo tim cook treasurer gary wipfler q session analyst please note information hear discussion today may consist forward looking statement regarding revenue gross margin operating expense income expense stock based compensation expense tax earnings actual result trend could differ materially forecast i...,"{'apple': 0.338, 'oppenheimer': 0.242, 'peter': 0.196, 'revenue': 0.187, 'thank': 0.162, 'december': 0.142, 'wa': 0.14, 'today': 0.139, 'form': 0.136, 'joining': 0.134, 'margin': 0.134, 'gross': 0.131, 'wipfler': 0.127, 'treasurer': 0.124, 'expense': 0.122, 'attached': 0.122, 'statement': 0.121, 'earnings': 0.119, 'gary': 0.113, 'breaking': 0.109}","[apple, oppenheimer, peter, revenue, thank, december, wa, today, form, joining, margin, gross, wipfler, treasurer, expense, attached, statement, earnings, gary, breaking]"
1,1,0.692090,0.206215,0.076271,0.025424,0,240,0,22,0,0,0.997073,0.002927,0.000172,0.999828,"approximately two times net income. I’d like to first talk about our Mac products and services, which represented 47% of total quarterly revenue. We are extremely pleased to have shipped 2.32 million Macs, exceeding the previous December quarter shipments by over 700,000 units and representing 44% year over year growth. That’s more than 2.5 times the overall market rate of growth for the Decem...",approximately two time net income like first talk mac product service represented total quarterly revenue extremely please shipped million mac exceeding previous december quarter shipment unit representing year year growth time overall market rate growth december quarter based latest forecast published idc sale new imac announced august continued robust helping drive year year growth desktop s...,"{'ipod': 0.498, 'leopard': 0.365, 'mac': 0.26, 'extremely': 0.145, 'music': 0.144, 'tiger': 0.138, 'began': 0.131, 'channel': 0.119, 'sold': 0.119, 'wifi': 0.117, 'december': 0.117, 'inventory': 0.113, 'mainstream': 0.113, 'reviewer': 0.108, 'ended': 0.108, 'total': 0.105, 'revenue': 0.103, 'million': 0.1, 'surpassing': 0.099, 'imac': 0.099}","[ipod, leopard, mac, extremely, music, tiger, began, channel, sold, wifi, december, inventory, mainstream, reviewer, ended, total, revenue, million, surpassing, imac]"
2,2,0.708333,0.213889,0.061111,0.016667,1,229,1,21,0,0,0.993504,0.006496,0.056833,0.943167,"mobile applications. Because of the higher cost associated with the large touch screen and more powerful processor required to run applications like Safari, this was the most expensive iPod we’ve brought to market for some time. So we had the challenge of establishing a completely new type of iPod at the top of the line at a price point above where we’ve been for quite some time, and we succee...",mobile application higher cost associated large touch screen powerful processor required run application like safari wa expensive ipod brought market time challenge establishing completely new type ipod top line price point quite time succeeded addition selling successfully ipod touch wa responsible overall increase ipod asp drove revenue year year last week announced major software upgrade ip...,"{'ipod': 0.495, 'iphon